### BERT Score로 입력문을 요약하는 모델이 생성한 요약문이 입력한 문장을 얼마나 잘 요약하는지 평가해보기
- 인코더로 입력 문장을 이해하는 Bert 모델을 활용해 두 문장이 의미적으로 유사한지 평가
- ex) "강아지가 뛰어놀고 있다" <-> "개의 활동성이 높다" 의 두 문장은 일치하는 단어는 없지만 의미적으로 유사하면 높은 점수 부여
- 평가 데이터 셋 : 한국어 기사 본문을 요약한 'daekeun-ml/naver-news-summarization-ko'의 test 데이터셋 중 100개를 추출하여 사용

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", max_length=128, model="edgeun/t5-small-korean-news-summarizer")

Device set to use cuda:0


In [ ]:
inputs = "summarize: " + """
달력은 봄에 가까워지고 있지만, 아직 두터운 겨울 외투 없이는 바깥에 나서기가 어렵습니다.
막바지 강추위는 이번 주말까지 이어지겠는데요.
서울의 아침 기온이 영하 7도 선을 유지하면서 평년 기온을 4도 가량 밑돌겠습니다.
특히, 한파특보가 내려진 일부 중부와 경북 지역은 영하 10도 안팎까지 내려가겠고, 전국에 바람이 강하게 불며 체감 온도를 뚝뚝 떨어트리겠습니다.
맑은 날씨가 이어지면서 건조특보 지역은 나날이 확대되고 있는데요.
동해안을 따라서는 건조경보가, 서울 등 중부와 동쪽 많은 지역에는 건조주의보가 내려졌습니다. 꺼진 불씨도 다시 한번 확인하시기 바랍니다.
아침 기온은 서울 영하 7도, 청주와 대구가 영하 5도로 오늘만큼 춥겠고요.
낮에도 서울은 2도, 광주는 3도에 머물겠습니다.
일요일에는 충남과 호남, 제주에 비나 눈이 내리겠고, 다음 주부터는 서서히 봄기운이 느껴지겠습니다.
"""

result = pipe(inputs)

In [ ]:
result[0]['generated_text']

'서울의 아침 기온이 영하 7도 선을 유지하면서 평년 기온을 4도 가량 밑돌겠고, 낮에도 서울은 2도, 광주는 3도에 머물겠고, 전국에 바람이 강하게 불며 체감 온도를 뚝뚝 떨어트리며, 낮에도 서울은 2도, 광주는 3도에 머물겠습니다.'

In [1]:
!pip install datasets -qqq
from datasets import load_dataset

test_dataset = load_dataset("daekeun-ml/naver-news-summarization-ko", split='test').shuffle(seed=42).select(range(100))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.7 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/787 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/66.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/7.45M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/8.17M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/22194 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2466 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2740 [00:00<?, ? examples/s]

In [2]:
test_dataset

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
    num_rows: 100
})

In [3]:
test_dataset[0]['document']

'요새 가장 뜨거운 화두는 환율인 듯합니다. 서울 외환시장에서 원 달러 환율이 1300원을 예사로 넘으면서 여기저기서 우려의 목소리가 나오죠. 달러를 기준통화 base currency 로 하는 우리 환율 표기상 환율이 오르면 그만큼 원화 가치가 낮아졌다는 뜻입니다. 그런데 가끔 이런 궁금증을 제기하는 분들이 계십니다. 1990년대 말 외환위기 전엔 달러당 700 800원 수준이었던 원 달러 환율이 왜 국민들이 더 잘 살게 되고 나가경제도 발전한 지금 1300원을 넘나드냐는 것이죠. 나라가 더 잘살게 됐는데 그 나라의 통화 가치는 왜 떨어졌냐는 의문입니다. 우리가 잘 살게 되면 그만큼 국가 경제도 튼튼해졌단 뜻이니 원화도 강해져야 맞는 것 아니냐는 것이죠. 일단 이는 나라 경제가 곤두박질치면서 화폐 가치가 폭락한 일부 나라들 아프리카 짐바브웨나 중남미 베네수엘라 같은 나라들 때문에 생긴 오해인 것 같습니다. 인터넷에서 판매하는 100조 짐바브웨 달러 지폐 연합 당장 내일 환율이 어떻게 될지는 환율 전략가가 아니라 뭐라고 말씀은 못 드리겠고 다만 장기적으로 환율이 어떻게 결정되는지 경제학 이론적으로 설명을 드릴까 합니다. 수학식이 계속 나오고 좀 재미가 없을 수 있습니다. 아래쪽에 결론만 보셔도 됩니다 가장 먼저 살펴볼 건 이자율평가설 Interest rate parity 입니다. Parity라는 건 우리말로 동등성 등가 등으로 해석이 되는데 서로 다른 두 변수 간 일정한 어떤 관계가 있다는 것으로 반드시 성립해야 하는 방정식 과는 다릅니다. X국의 통화가 기준통화고 Y국 통화가 상품통화 price currency 일 때 1년 뒤 선도금리는 F 와 현재 금리 S 의 관계는 아래와 같습니다. 그 차이 F S 가 바로 프리미엄으로 두 나라 간 명목 이자율 차이와 같습니다. 엄밀히 말하면 근사값입니다 예를 들어 Y국 이자율이 3%고 X국 이자율이 5%면 선도금리와 현재 금리 간 차이인 프리미엄은 2% 입니다. 근데 만약 기준통화인 X가 선물환 시장에서 2%만큼 프리미엄

In [ ]:
from huggingface_hub import login

login(token="****")

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_id = "edgeun/t5-small-korean-news-summarizer"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/788 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/383M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [6]:
# 모델이 지금 어디에 있는지 확인
model.device  # device(type='cpu') -> cpu에 모델이 있음

device(type='cpu')

In [7]:
# 모델 GPU로 보내기
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(50358, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50358, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [8]:
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(50358, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50358, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [9]:
from torch.utils.data import DataLoader

# DataLoader를 사용해서 배치 단위로 데이터 가져오기
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [12]:
from tqdm import tqdm

predictions = []
references = []

for batch in tqdm(test_dataloader):
    input_texts = ["summarize: " + doc for doc in batch["document"]]
    reference_texts = batch["summary"]

    # 배치 토큰화
    inputs = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # 입력 텐서를 GPU로 이동

    # 모델 추론 (배치 처리)
    output_ids = model.generate(**inputs, do_sample=True, max_length=128, num_beams=4, no_repeat_ngram_size=2)

    # 배치 디코딩
    batch_predictions = tokenizer.batch_decode(output_ids, skip_special_tokens=True)

    # 결과 저장
    predictions.extend(batch_predictions)
    references.extend(reference_texts)

100%|██████████| 13/13 [00:47<00:00,  3.66s/it]


In [13]:
predictions[:5]

['서울 외환시장에서 원 달러 환율이 1300원을 예사로 넘으면서 여기저기서 우려의 목소리가 나옵니다.',
 '금융권의 클라우드 도입이 궤도에 오르고 있는 가운데 안정적인 IT인프라 운영을 위한 금융 혁신 방안 를 주제로 경험과 공유할 계획이다.',
 '국내 연구진이 건축 자재로 쓰이는 폴리염화비닐 PVC 고분자 겔을 응용해 마찰전기 나노발전기를 구현하는 데 성공했으며, 3일 한국연구재단에 따르면 대구경북과학기술원 DGIST 이주혁 교수는 에너지 발전 소자로써 기계적 에너를 수확할 수 있을 뿐 아니라 전극 그리드 패턴 없이도 임의의 위치를',
 '산업통상자원부와 한국전력에 따르면 전기요금과 가스요금이 1일 함께 오르면서 물가 부담은 더 커질 전망이다.',
 '3일 정부서울청사에서 방기선 기획재정부 1차관 주재로 제2차 비상경제차관회의를 열고 물가 상승기 편승한 지역별 주유소 가격 담합 가짜석유 유통과 같은 불법행위가 엄단하고 인플레이션 압력이 계속 이어질 것이란 전망을 내놨다.']

In [15]:
!pip install bert_score -qqq
from bert_score import BERTScorer

# BERTScore 초기화
scorer = BERTScorer(model_type="klue/roberta-large", num_layers=24, batch_size=8, lang="ko", rescale_with_baseline=False, device=device)  # "klue/roberta-large": 한국어 모델

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.3 MB/s eta 0:00:00


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# Bert Score로 정밀도, 재현율, F1 스코어 계산
P, R, F1 = scorer.score(predictions, references)

In [17]:
P

tensor([0.7732, 0.8195, 0.8344, 0.8264, 0.7647, 0.8366, 0.7486, 0.6870, 0.9776,
        0.8896, 0.8932, 0.7561, 0.7800, 0.9229, 0.9040, 0.8413, 0.9207, 0.7146,
        0.9375, 0.8358, 0.8381, 0.7491, 0.9665, 0.6665, 0.8142, 0.7159, 0.8729,
        0.9455, 0.8620, 0.9583, 0.8977, 0.8121, 0.8493, 0.8011, 0.7865, 0.7745,
        0.7083, 0.9096, 0.9675, 0.8614, 0.9416, 0.9640, 0.8532, 0.9467, 0.9250,
        0.9283, 0.8677, 0.8595, 0.8974, 0.8021, 0.8596, 0.8153, 0.7398, 0.8027,
        0.8569, 0.8629, 0.7588, 0.8127, 0.7352, 0.8249, 0.9337, 0.7782, 0.7465,
        0.9708, 0.8959, 0.8284, 0.7859, 0.9400, 0.9049, 0.8070, 0.9432, 0.7860,
        0.6818, 0.8849, 0.7351, 0.8481, 0.7112, 0.9062, 0.9196, 0.8269, 0.7807,
        0.9134, 0.7817, 0.7328, 0.8758, 0.7831, 0.8602, 0.9214, 0.9674, 0.8505,
        0.7808, 0.9180, 0.8250, 0.8239, 0.8705, 0.9268, 0.8285, 0.7873, 0.7787,
        0.8820])

In [18]:
import pandas as pd

result = pd.DataFrame({
        '생성한 요약문' : predictions,
        '정밀도' : P.tolist(),
        '재현율' : R.tolist(),
        'F1' : F1.tolist() })

result.sort_values(by='F1', ascending=False)

,생성한 요약문,정밀도,재현율,F1
22,추경호 경제부총리 겸 기획재정부 장관이 3일 정부서울청사에서 열린 제3차 비상경제장...,0.966539,0.958942,0.962725
38,6일 오후 부산상공회의소 대강당에서 동남권관문공항추진위원회와 신항공추진범시민운동본부...,0.967548,0.904980,0.935219
70,삼성전자 홈캐스트는 삼성전기 베트남 법인과 스마트폰용 이미지센서 공급계약을 체결했다...,0.943158,0.907168,0.924813
78,한국우편사업진흥원 원장 민재석은 4일부터 17일까지 14일간 여름 제철 먹거리와 상...,0.919637,0.926250,0.922932
37,롯데바이오로직스는 1일 밀리포어 씨그마와의 이번 업무협약으로 차세대 치료제 등 바이...,0.909599,0.923934,0.916711
...,...,...,...,...
7,타다 서비스를 제공하는 VCNC에 따르면 총 80편 전체 18분 분량으로 제작된 이...,0.686996,0.659080,0.672749
72,6일 경기 고양시 킨텍스에서 열린 제20회 국제 나노기술심포지엄 및 융합전시회 20...,0.681832,0.661924,0.671731
23,8일 에스티팜은 지난 5월 북미 지역의 바이오텍과 mRNA 백신에 사용되는 지질나노...,0.666494,0.607477,0.635619
36,금융공기업이 입주한 부산국제금융센터 BIFC 전경.,0.708313,0.564580,0.628332


In [19]:
result.sort_values(by='F1', ascending=True)

,생성한 요약문,정밀도,재현율,F1
5,고승범 금융위원장이 5일 정부서울청사에서 취임식을 갖고 이임식 꽃다발을 받고 있다.,0.836619,0.484619,0.613730
36,금융공기업이 입주한 부산국제금융센터 BIFC 전경.,0.708313,0.564580,0.628332
23,8일 에스티팜은 지난 5월 북미 지역의 바이오텍과 mRNA 백신에 사용되는 지질나노...,0.666494,0.607477,0.635619
72,6일 경기 고양시 킨텍스에서 열린 제20회 국제 나노기술심포지엄 및 융합전시회 20...,0.681832,0.661924,0.671731
7,타다 서비스를 제공하는 VCNC에 따르면 총 80편 전체 18분 분량으로 제작된 이...,0.686996,0.659080,0.672749
...,...,...,...,...
37,롯데바이오로직스는 1일 밀리포어 씨그마와의 이번 업무협약으로 차세대 치료제 등 바이...,0.909599,0.923934,0.916711
78,한국우편사업진흥원 원장 민재석은 4일부터 17일까지 14일간 여름 제철 먹거리와 상...,0.919637,0.926250,0.922932
70,삼성전자 홈캐스트는 삼성전기 베트남 법인과 스마트폰용 이미지센서 공급계약을 체결했다...,0.943158,0.907168,0.924813
38,6일 오후 부산상공회의소 대강당에서 동남권관문공항추진위원회와 신항공추진범시민운동본부...,0.967548,0.904980,0.935219


In [20]:
print(f"평균 정밀도: {result['정밀도'].mean():.4}")
print(f"평균 재현율: {result['재현율'].mean():.4}")
print(f"평균 F1 스코어: {result['F1'].mean():.4}")

평균 정밀도: 0.842
평균 재현율: 0.7876
평균 F1 스코어: 0.8112
